In [1]:
import scanpy as sc
# import scvi
import pandas as pd
import math
import numpy as np
# Plotting

import seaborn as sns

# System
from pathlib import Path
import os

# Ensure you are always in the parent dir
os.chdir('/home/kyan/git/cv-scdl3991/')
# data_path = Path('data/MH/MH_raw_counts.csv')
data_path = Path('data/')
dir_path = Path('MSC/osmfish.h5ad')
# Warnings 

import warnings
warnings.simplefilter("ignore")

In [2]:
def setup_qc_metrics(adata):
    """
    Sets up and performs doublet detection for an AnnData object.

    1. Identifies the top 2000 highly variable genes using the 'seurat_v3' method 
    2. Uses the scVI model to detect and remove doublets
    3. Trains a SOLO model to further refine the detection of doublets.
   
    Parameters
    ----------
    adata : AnnData
        An AnnData object containing single-cell gene expression data.

    Returns
    -------
    AnnData
        The input AnnData object with additional QC metrics, including doublet predictions, stored in the `obs` DataFrame.
    """
    # setup

    adata.var_names_make_unique()

    sc.pp.highly_variable_genes(adata, n_top_genes = 2000, subset=True, flavor = 'seurat_v3', inplace=True)

    # # using scvi to remove doublets
    # scvi.model.SCVI.setup_anndata(adata)
    # vae = scvi.model.SCVI(adata)
    # vae.train()

    # # training a solo model
    # solo = scvi.external.SOLO.from_scvi_model(vae)
    # solo.train()

    # temp_df = solo.predict()
    # temp_df['predict'] = solo.predict(soft = False)
    # temp_df['difference'] = temp_df.doublet - temp_df.singlet 
    # doublets = temp_df[(temp_df['predict'] == 'doublet') & (temp_df['difference'] > 1)] # filtering out only those that have a predicted difference of more than 1

    # # adding doublet prediction to adata object
    # adata.obs['doublet'] = adata.obs.index.isin(doublets)

    return adata


def compute_qc_metrics(adata):
    """
    Compute quality control (QC) metrics for an AnnData object.

    This function calculates various QC metrics for the input AnnData object, such as the percentage of counts 
    assigned to specific gene categories (mitochondrial, ribosomal, hemoglobin genes), and the percentage of counts 
    assigned to the top x% of genes per cell. Additionally, it computes the percentage of highly variable genes.

    Parameters
    ----------
    adata : AnnData
        An AnnData object containing single-cell gene expression data. The input object should have a `var` DataFrame 
        with gene names.

    Returns
    -------
    AnnData
        The input AnnData object with additional QC metrics stored in the `obs` and `var` DataFrames.
    """
    
    # default is no filtering 
    # sc.pp.filter_cells(adata, min_genes = 200) 
    
    adata.var["mt"] = adata.var_names.str.startswith(("MT-", "mt-", "Mt-"))
    adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
    adata.var["hb"] = adata.var_names.str.contains(("^HB[^(P)]"))


    # To calculate the % of counts assigned to the top x genes per cell. Some data have very small cell counts so % were used instead
    n_genes = len(adata.var)
    percents = [0.01, 0.05, 0.1, 0.2] # 1% , 5%, 10%, 20% of genes)
    ceiling_values = [math.ceil(n_genes * p) for p in percents]
    sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=ceiling_values, log1p=True)

    # Doing some renaming
    percent_names = [f"pct_counts_in_top_{n}_genes" for n in ceiling_values]
    new_names = [f"pct_counts_in_top_{round(n*100)}_pct_genes" for n in percents]
    adata.obs.rename(columns = dict(zip(percent_names, new_names)), inplace = True)

    rename_totals = {'total_counts': 'total_counts_genes', 'log1p_total_counts': 'log1p_total_counts_genes'}
    adata.var.rename(columns = rename_totals, inplace = True)
    # remove = ['total_counts_mt', 'log1p_total_counts_mt', 'total_counts_ribo', 
    #       'log1p_total_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb']
    
    # adata.obs = adata.obs[[x for x in adata.obs.columns if x not in remove]]
    
    #### Other Metrics #####
    # adata.var['pct_highly_var_genes'] = adata.var["highly_variable"].sum() / len(adata.var). Realised this is not a useful metric
    
    return adata

def extract_metrics(adata, agg=np.median, exclude = None):
    """
    Extracts summary statistics from an AnnData object.
    
    Parameters:
    - adata: AnnData object, after highly variable genes, and doublets have been identified.
    - agg: function to apply to each variable (default: np.mean)
    
    Returns:
    - dict, column_names: dictionary of {row: [metrics]}, list of all column_names
    """
    try:
        index = adata.uns['name']
    except:
        print("No name set for AnnData object")

    ##### Computing metrics for all cells. #####

    # selecting only numeric quantities
    numeric_obs = adata.obs.select_dtypes(include=['number']).columns.tolist()
    numeric_obs = [item for item in numeric_obs if not item.startswith("_")]

    # applying our agg function
    obs_metrics = adata.obs[numeric_obs].apply(agg).to_list()

    ##### Computing metrics for all genes.#####
    
    # selecting only numeric quantities
    numeric_vars = adata.var.select_dtypes(include=['number']).columns.tolist()

    # EXCLUDE
    exclude = ['highly_variable_rank']
    
    numeric_vars = [item for item in numeric_vars if ((not item.startswith("_")) and (item not in exclude))]

    # applying our agg function
    var_metrics = adata.var[numeric_vars].apply(agg).to_list()

    ##### Custom defined metrics #####

    ## TODO

    ##### PCA Extraction #####

    obs_metrics.extend(var_metrics)
    numeric_obs.extend(numeric_vars)
    
    return {index: obs_metrics}, numeric_obs

In [3]:
def run_all_qc_partially(adata):
    save_path = "outputs/characteristics/characteristics.csv"
    
    adata = setup_qc_metrics(adata)
    adata = compute_qc_metrics(adata)
    dataset_row, columns = extract_metrics(adata)
    new_df = pd.DataFrame.from_dict(dataset_row, columns = columns, 
                                    orient='index') 
    return new_df

In [4]:
def run_all_qc(adata):
    save_path = "outputs/characteristics/characteristics.csv"
    
    adata = setup_qc_metrics(adata)
    adata = compute_qc_metrics(adata)
    dataset_row, columns = extract_metrics(adata)


    ### Adding onto the end of new df
    saved_df = pd.read_csv(save_path, index_col = 0)
    new_df = pd.DataFrame.from_dict(dataset_row, columns = columns, 
                                    orient='index') 
    print(f"Adding entry < {new_df.index} to dataset")

    ### TESTING FOR DUPLICATED COLUMNS
    duplicate_columns = new_df.columns[new_df.columns.duplicated()]
    # Display results
    if duplicate_columns.any():
        print(f"WARNING: Duplicate columns in {new_df.index}:")
        print(duplicate_columns)

    common_columns = saved_df.columns.intersection(new_df.columns)

    print(f"Old Columns: {saved_df.columns} \n")
    print(f"New Columns: {new_df.columns} \n")
    print(f"Common Columns: {common_columns} \n")
    
    # Select only the common columns from both DataFrames
    saved_df_common = saved_df[common_columns]
    new_df_common = new_df[common_columns]

    # Append df2 to df1
    result = pd.concat([saved_df_common, new_df_common], join="inner")
    result.to_csv(save_path, index= True)

    print(f"Saved to disk at path: {save_path}")

## MSC

In [5]:
data_path_MSC = Path('data/MSC/MSC_gene_expression_FINAL.h5ad')
adata = sc.read_h5ad(data_path_MSC)
adata.uns['name'] = 'MSC'

In [6]:
adata = setup_qc_metrics(adata)

In [7]:
adata = compute_qc_metrics(adata) # this can only be run once

In [8]:
dataset_row, columns = extract_metrics(adata)

In [9]:
characteristics_df = pd.DataFrame.from_dict(dataset_row, columns = columns, orient='index') # 34 features so far
characteristics_df

,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_1_pct_genes,pct_counts_in_top_5_pct_genes,pct_counts_in_top_10_pct_genes,pct_counts_in_top_20_pct_genes,total_counts_mt,log1p_total_counts_mt,...,pct_counts_hb,means,variances,variances_norm,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts_genes,log1p_total_counts_genes
MSC,23.0,3.178054,5328.0,8.580919,20.125708,33.299927,51.226799,68.549191,0.0,0.0,...,0.0,159.733532,59376.838508,0.99151,3379.0,159.733551,5.079748,30.171523,772950.625,13.557972


In [10]:
save_path = "outputs/characteristics/characteristics.csv"
characteristics_df.to_csv(save_path, index=True)

## Loading in all other data

In [7]:
data_path_MSC = Path('data/MSC/MSC_gene_expression_FINAL.h5ad')
data_path_DPLC_1 = Path('data/DLPC/151507')
data_path_DPLC_2 = Path('data/DLPC/151508')
data_path_DPLC_3 = Path('data/DLPC/151509')
data_path_HBCA1 = Path('data/HBCA1/')

adata = sc.read_h5ad(data_path_MSC)
adata.uns['name'] = 'MSC'


adata1 = sc.read_visium(data_path_DPLC_1,
                       count_file = "151507_filtered_feature_bc_matrix.h5")
adata1.uns['name'] = 'DPLC_151707'

adata2 = sc.read_visium(data_path_DPLC_2,
                       count_file = "151508_filtered_feature_bc_matrix.h5")
adata2.uns['name'] = 'DPLC_151708'


adata3 = sc.read_visium(data_path_DPLC_3,
                       count_file = "151509_filtered_feature_bc_matrix.h5")
adata3.uns['name'] = 'DPLC_151709'


adata4 = sc.read_visium(data_path_HBCA1,
                       count_file = "V1_Breast_Cancer_Block_A_Section_1_filtered_feature_bc_matrix.h5")
adata4.uns['name'] = 'HBCA1'

In [12]:
adatas = [adata1, adata2, adata3, adata4]

In [8]:
run_all_qc(adata1)

Adding entry < Index(['DPLC_151707'], dtype='object') to dataset
Old Columns: Index(['n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts',
       'log1p_total_counts', 'pct_counts_in_top_1_pct_genes',
       'pct_counts_in_top_5_pct_genes', 'pct_counts_in_top_10_pct_genes',
       'pct_counts_in_top_20_pct_genes', 'total_counts_mt',
       'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo',
       'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb',
       'log1p_total_counts_hb', 'pct_counts_hb', 'means', 'variances',
       'variances_norm', 'n_cells_by_counts', 'mean_counts',
       'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts_genes',
       'log1p_total_counts_genes'],
      dtype='object') 

New Columns: Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts',
       'pct_counts_in_top_1_pct_genes', 'pct_counts_in_top_5_pct_genes',
       'pct_counts_in

In [14]:
for adata in [adata3, adata4]:
    try:
        run_all_qc(adata)
    except:
        print(f"{adata.uns['name']} failed. Proceeding with the next")

Adding entry < Index(['DPLC_151709'], dtype='object') to dataset
Old Columns: Index(['n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts',
       'log1p_total_counts', 'pct_counts_in_top_1_pct_genes',
       'pct_counts_in_top_5_pct_genes', 'pct_counts_in_top_10_pct_genes',
       'pct_counts_in_top_20_pct_genes', 'total_counts_mt',
       'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo',
       'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb',
       'log1p_total_counts_hb', 'pct_counts_hb', 'means', 'variances',
       'variances_norm', 'n_cells_by_counts', 'mean_counts',
       'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts_genes',
       'log1p_total_counts_genes'],
      dtype='object') 

New Columns: Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts',
       'pct_counts_in_top_1_pct_genes', 'pct_counts_in_top_5_pct_genes',
       'pct_counts_in

In [16]:
adata4.var_names.to_list()[-20:]

['NDUFA1',
 'GRIA3',
 'SH2D1A',
 'APLN',
 'SASH3',
 'GPC4',
 'GPC3',
 'MIR503HG',
 'MCF2',
 'LINC00632',
 'CXorf40A',
 'HMGB3',
 'AC244102.4',
 'GABRQ',
 'BGN',
 'PLXNA3',
 'LAGE3',
 'AC234781.1',
 'MT-ATP8',
 'MT-ND6']

In [5]:
data_path_MH = Path('data/MH')
def load_MH_datasets(MH_dir):
    datasets = []
    # Files must follow the naming scheme MH_{sample}.h5ad
    for file in MH_dir.glob("MH_*.h5ad"):
        print(f"Loading data from {file.stem}...")
        adata = sc.read_h5ad(file)
        adata.uns['name'] = file.stem
        datasets.append(adata)
    return datasets

In [ ]:
res1 = load_MH_datasets(data_path_MH)

In [7]:
test = res[1]

In [8]:
setup_qc_metrics(test)

AnnData object with n_obs × n_vars = 5319 × 135
    obs: 'Cell_ID', 'Animal_ID', 'Animal_sex', 'Behavior', 'Bregma', 'Cell_class', 'Neuron_cluster_ID', 'samples', 'X', 'Y'
    var: 'Gene', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'X_name', 'name', 'hvg'

In [9]:
compute_qc_metrics(test)

AnnData object with n_obs × n_vars = 5319 × 135
    obs: 'Cell_ID', 'Animal_ID', 'Animal_sex', 'Behavior', 'Bregma', 'Cell_class', 'Neuron_cluster_ID', 'samples', 'X', 'Y', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_1_pct_genes', 'pct_counts_in_top_5_pct_genes', 'pct_counts_in_top_10_pct_genes', 'pct_counts_in_top_20_pct_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb'
    var: 'Gene', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts_genes', 'log1p_total_counts_genes'
    uns: 'X_name', 'name', 'hvg'

In [13]:
test.uns["name"]

'MH_5'